In [ ]:
import tkinter as tk
from tkinter_itk import ITK_viewer
import SimpleITK as sitk
import asyncio

# https://gist.github.com/sankhakarfa/35e522c31677fbc20b0e165cb05f9b41
async def run_tk(root, interval=0.1):
    '''
    Run a tkinter app in an asyncio event loop.
    '''
    try:
        while True:
            
            await root.update()
            await asyncio.sleep(interval)
    except tk.TclError as e:
        if "application has been destroyed" not in e.args[0]:
            raise

root = tk.Tk()
async def main(root):
    app = ITK_viewer.MainWindow(root)
    await run_tk(app)

loop = asyncio.get_running_loop()
loop.create_task(main(root))
await asyncio.sleep(5)
mainwindow = root._nametowidget('!mainwindow')


In [ ]:
def get_dummy_SITK_image():
    """placeholder"""
    image = sitk.Image(128,128,50, sitk.sitkInt16)
    number = 10
    for i in range(int(image.GetSize()[0]/2)):
        image[i*2, :,:] = number
        number += 3
    
    for i in range(int(image.GetSize()[1]/2)):
        image[:, i*2, :] += number
        number += 3
    image.SetSpacing([2,3,1])
    image.SetOrigin([6,0.15,0])
    image.SetDirection([1,0.25,0,0,0.75,0,0,0,1])
    return image

In [ ]:
random_itk_image = get_dummy_SITK_image()
mainwindow.new_image_input(random_itk_image, "example_name", add=True)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

np_ITK_image = sitk.GetArrayViewFromImage(random_itk_image)

"""plot image"""
plt.imshow(np_ITK_image[5,:,:], cmap='gray')
plt.show()


In [ ]:
#histogram
plt.hist(np_ITK_image.flatten(), bins=50)
plt.show()


In [ ]:
#creating example segmentation
def get_dummy_SITK_image_segmentation():
    """placeholder"""
    import cv2
    size = random_itk_image.GetSize()
    image = sitk.Image([size[-1], size[0], size[1]], sitk.sitkUInt8)

    # Get the number of slices
    num_slices = image.GetSize()[0]

    # Create an empty list to store the modified slices
    slices_with_numbers = []
    np_array = np.empty(image.GetSize(), dtype=np.int16)
    # Loop over each slice
    for i in range(num_slices):
        # Extract the slice
        
        # Normalize the slice array for visualization
        np_array[i, :, :] = cv2.normalize(np_array[i, :, :], None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        
        # Write the slice number on the slice
        cv2.putText(np_array[i, :, :], f"{i}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, 5, 2)

    new_img = sitk.GetImageFromArray(np_array)
    return new_img

In [ ]:
mainwindow.load_segmentation(get_dummy_SITK_image_segmentation(), "example_name")
